In [1]:
import numpy as np
import sys
from urllib.request import urlopen
import urllib
import pandas as pd
Cleveland_data_URL = 'http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.hungarian.data'
np.set_printoptions(threshold=sys.maxsize) 

In [2]:
names = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang','oldpeak','slope','ca','thal',
'heartdisease']
heartDisease = pd.read_csv(urlopen(Cleveland_data_URL), names = names)
heartDisease.head()
del heartDisease['ca']
del heartDisease['slope']
del heartDisease['thal']
del heartDisease['oldpeak']

In [3]:
heartDisease = heartDisease.replace('?', np.nan)
heartDisease.dtypes
heartDisease.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'heartdisease'],
      dtype='object')

In [6]:
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator

In [14]:
model = BayesianModel([('age','trestbps'), ('age','fbs'), ('sex', 'trestbps'), ('sex', 'trestbps'),('exang','trestbps'),('trestbps','heartdisease'),('fbs','heartdisease'),('heartdisease','restecg'),('heartdisease','thalach'),('heartdisease','chol')])                                                                                                        

In [20]:
model.fit(heartDisease, estimator=MaximumLikelihoodEstimator)
print(model.get_cpds('age'))
print(model.get_cpds('chol'))
print(model.get_cpds('sex'))
model.get_independencies()

+---------+------------+
| age(28) | 0.00383142 |
+---------+------------+
| age(29) | 0.00383142 |
+---------+------------+
| age(30) | 0.00383142 |
+---------+------------+
| age(31) | 0.00766284 |
+---------+------------+
| age(32) | 0.0153257  |
+---------+------------+
| age(33) | 0.00766284 |
+---------+------------+
| age(34) | 0.0153257  |
+---------+------------+
| age(35) | 0.0191571  |
+---------+------------+
| age(36) | 0.0191571  |
+---------+------------+
| age(37) | 0.0306513  |
+---------+------------+
| age(38) | 0.0191571  |
+---------+------------+
| age(39) | 0.0344828  |
+---------+------------+
| age(40) | 0.0191571  |
+---------+------------+
| age(41) | 0.0383142  |
+---------+------------+
| age(42) | 0.0268199  |
+---------+------------+
| age(43) | 0.0421456  |
+---------+------------+
| age(44) | 0.0268199  |
+---------+------------+
| age(45) | 0.0229885  |
+---------+------------+
| age(46) | 0.045977   |
+---------+------------+
| age(47) | 0.0344828  |


(age _|_ exang, sex)
(age _|_ sex | exang)
(age _|_ exang | sex)
(age _|_ exang, sex | fbs)
(age _|_ restecg, thalach, chol | heartdisease)
(age _|_ sex | fbs, exang)
(age _|_ restecg, thalach, chol | exang, heartdisease)
(age _|_ restecg, chol | thalach, heartdisease)
(age _|_ exang | fbs, sex)
(age _|_ restecg, thalach, chol | heartdisease, sex)
(age _|_ restecg, thalach, chol, heartdisease | trestbps, fbs)
(age _|_ restecg, thalach, chol | trestbps, heartdisease)
(age _|_ restecg, thalach, chol | fbs, heartdisease)
(age _|_ restecg, thalach | chol, heartdisease)
(age _|_ thalach, chol | restecg, heartdisease)
(age _|_ restecg, chol | exang, thalach, heartdisease)
(age _|_ restecg, thalach, chol | exang, heartdisease, sex)
(age _|_ restecg, thalach, chol, heartdisease | trestbps, exang, fbs)
(age _|_ restecg, thalach, chol | trestbps, exang, heartdisease)
(age _|_ restecg, thalach, chol | fbs, exang, heartdisease)
(age _|_ restecg, thalach | exang, chol, heartdisease)
(age _|_ thalac

In [16]:
from pgmpy.inference import VariableElimination
HeartDisease_infer = VariableElimination(model)

In [22]:
q = HeartDisease_infer.query(variables=['heartdisease'], evidence={'age': 28})
print(q)
q = HeartDisease_infer.query(variables=['heartdisease'], evidence={'chol': 100})
print(q)

Eliminating: trestbps:   0%|          | 0/7 [00:00<?, ?it/s]

+-----------------+---------------------+
| heartdisease    |   phi(heartdisease) |
+=================+=====================+
| heartdisease(0) |              0.6390 |
+-----------------+---------------------+
| heartdisease(1) |              0.3610 |
+-----------------+---------------------+


Eliminating: sex: 100%|██████████| 7/7 [00:00<00:00, 104.52it/s]

+-----------------+---------------------+
| heartdisease    |   phi(heartdisease) |
+=================+=====================+
| heartdisease(0) |              1.0000 |
+-----------------+---------------------+
| heartdisease(1) |              0.0000 |
+-----------------+---------------------+
